In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd

In [106]:
## load baseline simulated data
simData = pd.read_csv('../DSSG2023-Heating-Loads-Data/heatPumpCal/results_BoroughProjs.csv')
simData['Census_Area_LowerCase'] = simData['Census_Area'].str.lower()

In [107]:
simData.head()

,Unnamed: 0,City,Exist_Fuel_Type,Scenario,Rebate_dol,Fuel_Esc_Rate,Temp_Projection,Census_Area,ANCSA_Region,Util_Name,...,CO2_lbs_saved,CO2_driving_miles_saved,Fuel_Use_Chg,Fuel_Price_Incremental,Elec_Use_Chg,Elec_Rate_Incremental,Elec_Rate_Avg_Base,Econ,Exist_Fuel,Census_Area_LowerCase
0,0,Adak,1,Base,0,0.03,0.0,Aleutians West Census Area,Aleut,Adak -TDX Residential,...,10628.423185,11917.022775,0.0,NaN,-7085.615457,1.315811,1.051414,Economic,Electricity,aleutians west census area
1,1,Adak,1,Base,0,0.06,0.0,Aleutians West Census Area,Aleut,Adak -TDX Residential,...,10628.423185,11917.022775,0.0,NaN,-7085.615457,1.315811,1.051414,Economic,Electricity,aleutians west census area
2,2,Adak,1,Base,0,0.09,0.0,Aleutians West Census Area,Aleut,Adak -TDX Residential,...,10628.423185,11917.022775,0.0,NaN,-7085.615457,1.315811,1.051414,Economic,Electricity,aleutians west census area
3,3,Adak,1,Base,2000,0.03,0.0,Aleutians West Census Area,Aleut,Adak -TDX Residential,...,10628.423185,11917.022775,0.0,NaN,-7085.615457,1.315811,1.051414,Economic,Electricity,aleutians west census area
4,4,Adak,1,Base,2000,0.06,0.0,Aleutians West Census Area,Aleut,Adak -TDX Residential,...,10628.423185,11917.022775,0.0,NaN,-7085.615457,1.315811,1.051414,Economic,Electricity,aleutians west census area


In [108]:
## loading heat fule type data
heatData = pd.read_excel('../DSSG2023-Heating-Loads-Data/Borough Heating Prop.xlsx')

In [109]:
heatData.head()

,Census_Area,Natural gas,Propane,Electricity,Oil
0,Aleutians East Borough,0.003623,0.009662,0.021739,0.964976
1,Aleutians West Census Area,0.012357,0.006178,0.060900,0.920565
2,Anchorage municipality,0.804394,0.010960,0.157433,0.027214
3,Bethel Census Area,0.002978,0.009680,0.020849,0.966493
4,Bristol Bay Borough,0.019293,0.000000,0.003215,0.977492


In [110]:
## loading places data
places = gpd.read_file('../DSSG2023-Heating-Loads-Data/shapefiles/Places2010/Places.shp')

In [111]:
## selecting specific columns
places = places[['NAME', 'HOUSEUNITS']]

In [112]:
places.head()

,NAME,HOUSEUNITS
0,Sand Point city,290.0
1,King Cove city,229.0
2,False Pass city,40.0
3,Cold Bay city,82.0
4,Akutan city,44.0


In [113]:
## loading borough shapefiles
boroughs = gpd.read_file('../DSSG2023-Heating-Loads-Data/shapefiles/Boroughs2020/')

In [114]:
## load Alan's city csv
city_meta = pd.read_csv('../DSSG2023-Heating-Loads-Data/heatPumpCal/city.csv')

In [115]:
city_meta = city_meta[['Name', 'census_city', 'census_area']]

In [116]:
city_meta.head()

,Name,census_city,census_area
0,Anchorage,Anchorage municipality,Anchorage municipality
1,Adak,Adak city,Aleutians West Census Area
2,Akutan,Akutan city,Aleutians East Borough
3,Allakaket,Allakaket city,Yukon-Koyukuk Census Area
4,Angoon,Angoon city,Hoonah-Angoon Census Area


## Find the place (city)'s house unit, and calculate the weighted average of outcomes by houseunit in a borough

In [117]:
## rename places shapefile
places.rename(columns={'NAME':'census_city'}, inplace=True)

In [118]:
## make sure all Alan's cities can be matched perfectly with census cities
city_meta['census_city'].isin(places['census_city']).mean()

1.0

In [119]:
## merge houseunits data to Alan's cities
city_meta_wHU = city_meta.merge(places, on='census_city')

In [120]:
city_meta_wHU.rename(columns={'Name':'City'}, inplace=True)

In [121]:
## merge houseunit data with simulated data
simData_wHU = simData.merge(city_meta_wHU, on='City')

In [122]:
## notice the discrepancy between Alan's cities (277) and census cities (255)
print([len(simData_wHU['census_city'].unique()), len(simData_wHU['City'].unique())])


[255, 277]


In [64]:
## average out across unique census cities
simData_uniqueCities = simData_wHU.groupby(['census_city', 'census_area', 'Exist_Fuel', 'Rebate_dol', 'Fuel_Esc_Rate', 'Temp_Projection'])[['HOUSEUNITS', "NPV", "CO2_lbs_saved", "CO2_driving_miles_saved", "Heating_Days_Above5", "Heating_Days_Below5"]].mean()

In [123]:
simData_uniqueCities.reset_index(inplace=True)

In [124]:
## get unique city-borough dataframe with houseunits
census_city_area_wHU = city_meta_wHU.drop(['City'], axis=1).drop_duplicates()

In [127]:
census_city_area_wHU.sort_values('census_area').head()

,census_city,census_area,HOUSEUNITS
148,Nelson Lagoon CDP,Aleutians East Borough,32.0
7,Akutan city,Aleutians East Borough,44.0
146,False Pass city,Aleutians East Borough,40.0
147,King Cove city,Aleutians East Borough,229.0
149,Sand Point city,Aleutians East Borough,290.0


In [146]:
## find the houseunit proportion (out of a borough) for a given city
HU_prop_df = pd.DataFrame()
for u in df['census_area'].unique():
    sub_df = df[df.census_area== u]
    sub_list = []
    for i in range(0,len(sub_df)):
        sub_list.append(sub_df.iloc[i]["HOUSEUNITS"]/sum(sub_df["HOUSEUNITS"]))
    sub_df2 = sub_df.copy()
    sub_df2["HU_prop"] = sub_list
    HU_prop_df = pd.concat([HU_prop_df,sub_df2])

In [147]:
HU_prop_df

,census_city,census_area,HOUSEUNITS,HU_prop
0,Anchorage municipality,Anchorage municipality,113032.0,1.000000
6,Adak city,Aleutians West Census Area,500.0,0.260010
12,Atka city,Aleutians West Census Area,43.0,0.022361
150,Nikolski CDP,Aleutians West Census Area,23.0,0.011960
151,St. George city,Aleutians West Census Area,61.0,0.031721
...,...,...,...,...
264,Port Lions city,Kodiak Island Borough,113.0,0.042966
265,Fairbanks city,Fairbanks North Star Borough,13056.0,0.827062
272,North Pole city,Fairbanks North Star Borough,916.0,0.058026
273,Salcha CDP,Fairbanks North Star Borough,585.0,0.037058


In [148]:
HU_prop_df.sort_values('census_area').to_csv('../Output/HU_prop_df.csv')

In [156]:
## merge the proportion w/ simulated data
simData_uniqueCities_wHUprop = simData_uniqueCities.merge(HU_prop_df.drop(['census_area'], axis=1), on ='census_city')

## Getting borough-level estimates weighted by city-houseunit-proportion

In [208]:
simData_uniqueCities_wHUprop['NPV_weighted'] = simData_uniqueCities_wHUprop['NPV'] * simData_uniqueCities_wHUprop['HU_prop']
simData_uniqueCities_wHUprop['CO2_lbs_saved_weighted'] = simData_uniqueCities_wHUprop['CO2_lbs_saved'] * simData_uniqueCities_wHUprop['HU_prop']
simData_uniqueCities_wHUprop['CO2_driving_miles_saved_weighted'] = simData_uniqueCities_wHUprop['CO2_driving_miles_saved'] * simData_uniqueCities_wHUprop['HU_prop']
simData_uniqueCities_wHUprop['Heating_Days_Above5_weighted'] = simData_uniqueCities_wHUprop['Heating_Days_Above5'] * simData_uniqueCities_wHUprop['HU_prop']
simData_uniqueCities_wHUprop['Heating_Days_Below5_weighted'] = simData_uniqueCities_wHUprop['Heating_Days_Below5'] * simData_uniqueCities_wHUprop['HU_prop']

In [209]:
borough_estimates_weightedByHU = simData_uniqueCities_wHUprop.groupby(['census_area', 'Exist_Fuel', 'Rebate_dol', 'Fuel_Esc_Rate', 'Temp_Projection'])[["NPV_weighted", "CO2_lbs_saved_weighted", "CO2_driving_miles_saved_weighted", "Heating_Days_Above5_weighted", "Heating_Days_Below5_weighted"]].sum()

In [210]:
borough_estimates_weightedByHU.reset_index(inplace=True)

In [211]:
borough_estimates_weightedByHU

,census_area,Exist_Fuel,Rebate_dol,Fuel_Esc_Rate,Temp_Projection,NPV_weighted,CO2_lbs_saved_weighted,CO2_driving_miles_saved_weighted,Heating_Days_Above5_weighted,Heating_Days_Below5_weighted
0,Aleutians East Borough,#1 Oil,0,0.03,0.0,-1114.817367,572.931347,642.394059,104.954833,0.000000
1,Aleutians East Borough,#1 Oil,0,0.03,1.5,-961.837917,558.069707,625.730581,104.951067,0.000000
2,Aleutians East Borough,#1 Oil,0,0.03,3.0,-887.967795,537.637642,602.821314,104.951067,0.000000
3,Aleutians East Borough,#1 Oil,0,0.06,0.0,-201.489759,572.931347,642.394059,104.954833,0.000000
4,Aleutians East Borough,#1 Oil,0,0.06,1.5,-17.543064,558.069707,625.730581,104.951067,0.000000
...,...,...,...,...,...,...,...,...,...,...
2425,Yukon-Koyukuk Census Area,Propane,4000,0.06,1.5,17313.429016,0.714701,0.801352,9.637317,2.106059
2426,Yukon-Koyukuk Census Area,Propane,4000,0.06,3.0,16996.397392,0.495319,0.555371,9.674077,1.905802
2427,Yukon-Koyukuk Census Area,Propane,4000,0.09,0.0,21615.855622,-0.098231,-0.110140,9.569975,2.272564
2428,Yukon-Koyukuk Census Area,Propane,4000,0.09,1.5,21181.959975,0.714701,0.801352,9.637317,2.106059


In [212]:
borough_estimates_weightedByHU.Exist_Fuel.unique()

array(['#1 Oil', 'Electricity', 'Propane', 'Natural Gas'], dtype=object)

## Weighting the borough estimates by fuel use distribution

In [213]:
heatData.rename(columns={'Census_Area':'census_area', 'Natural gas':'Natural Gas', 'Oil':'#1 Oil'}, inplace=True)

In [214]:
heatData.head()

,census_area,Natural Gas,Propane,Electricity,#1 Oil
0,Aleutians East Borough,0.003623,0.009662,0.021739,0.964976
1,Aleutians West Census Area,0.012357,0.006178,0.060900,0.920565
2,Anchorage municipality,0.804394,0.010960,0.157433,0.027214
3,Bethel Census Area,0.002978,0.009680,0.020849,0.966493
4,Bristol Bay Borough,0.019293,0.000000,0.003215,0.977492


In [215]:
borough_estimates_weightedByHU.head()

,census_area,Exist_Fuel,Rebate_dol,Fuel_Esc_Rate,Temp_Projection,NPV_weighted,CO2_lbs_saved_weighted,CO2_driving_miles_saved_weighted,Heating_Days_Above5_weighted,Heating_Days_Below5_weighted
0,Aleutians East Borough,#1 Oil,0,0.03,0.0,-1114.817367,572.931347,642.394059,104.954833,0.0
1,Aleutians East Borough,#1 Oil,0,0.03,1.5,-961.837917,558.069707,625.730581,104.951067,0.0
2,Aleutians East Borough,#1 Oil,0,0.03,3.0,-887.967795,537.637642,602.821314,104.951067,0.0
3,Aleutians East Borough,#1 Oil,0,0.06,0.0,-201.489759,572.931347,642.394059,104.954833,0.0
4,Aleutians East Borough,#1 Oil,0,0.06,1.5,-17.543064,558.069707,625.730581,104.951067,0.0


In [216]:
heatData_long = pd.melt(heatData, id_vars='census_area', value_vars=['Natural Gas', 'Propane','Electricity', '#1 Oil'], var_name='Exist_Fuel', value_name='fuel_prop')

In [217]:
borough_estimates_wFuelProp = borough_estimates_weightedByHU.merge(heatData_long, on=['census_area', 'Exist_Fuel'])

In [219]:
borough_estimates_wFuelProp.head()

,census_area,Exist_Fuel,Rebate_dol,Fuel_Esc_Rate,Temp_Projection,NPV_weighted,CO2_lbs_saved_weighted,CO2_driving_miles_saved_weighted,Heating_Days_Above5_weighted,Heating_Days_Below5_weighted,fuel_prop
0,Aleutians East Borough,#1 Oil,0,0.03,0.0,-1114.817367,572.931347,642.394059,104.954833,0.0,0.964976
1,Aleutians East Borough,#1 Oil,0,0.03,1.5,-961.837917,558.069707,625.730581,104.951067,0.0,0.964976
2,Aleutians East Borough,#1 Oil,0,0.03,3.0,-887.967795,537.637642,602.821314,104.951067,0.0,0.964976
3,Aleutians East Borough,#1 Oil,0,0.06,0.0,-201.489759,572.931347,642.394059,104.954833,0.0,0.964976
4,Aleutians East Borough,#1 Oil,0,0.06,1.5,-17.543064,558.069707,625.730581,104.951067,0.0,0.964976


In [220]:
borough_estimates_wFuelProp['NPV_weighted2'] = borough_estimates_wFuelProp['NPV_weighted'] * borough_estimates_wFuelProp['fuel_prop']
borough_estimates_wFuelProp['CO2_lbs_saved_weighted2'] = borough_estimates_wFuelProp['CO2_lbs_saved_weighted'] * borough_estimates_wFuelProp['fuel_prop']
borough_estimates_wFuelProp['CO2_driving_miles_saved_weighted2'] = borough_estimates_wFuelProp['CO2_driving_miles_saved_weighted'] * borough_estimates_wFuelProp['fuel_prop']
borough_estimates_wFuelProp['Heating_Days_Above5_weighted2'] = borough_estimates_wFuelProp['Heating_Days_Above5_weighted'] * borough_estimates_wFuelProp['fuel_prop']
borough_estimates_wFuelProp['Heating_Days_Below5_weighted2'] = borough_estimates_wFuelProp['Heating_Days_Below5_weighted'] * borough_estimates_wFuelProp['fuel_prop']

In [221]:
borough_estimates_wFuelProp.columns

Index(['census_area', 'Exist_Fuel', 'Rebate_dol', 'Fuel_Esc_Rate',
       'Temp_Projection', 'NPV_weighted', 'CO2_lbs_saved_weighted',
       'CO2_driving_miles_saved_weighted', 'Heating_Days_Above5_weighted',
       'Heating_Days_Below5_weighted', 'fuel_prop', 'NPV_weighted2',
       'CO2_lbs_saved_weighted2', 'CO2_driving_miles_saved_weighted2',
       'Heating_Days_Above5_weighted2', 'Heating_Days_Below5_weighted2'],
      dtype='object')

In [223]:
borough_estimates_weightedByHuFuel = borough_estimates_wFuelProp.groupby(['census_area', 'Rebate_dol', 'Fuel_Esc_Rate', 'Temp_Projection'])[['NPV_weighted2', 'CO2_lbs_saved_weighted2', 'CO2_driving_miles_saved_weighted2', 'Heating_Days_Above5_weighted2', 'Heating_Days_Below5_weighted2']].sum()

In [230]:
borough_estimates_weightedByHuFuel.reset_index(inplace=True)

In [235]:
borough_estimates_weightedByHuFuel['Heating_Days_Covered'] = borough_estimates_weightedByHuFuel['Heating_Days_Above5_weighted2'] / (borough_estimates_weightedByHuFuel['Heating_Days_Above5_weighted2'] + borough_estimates_weightedByHuFuel['Heating_Days_Below5_weighted2'])

In [236]:
borough_estimates_weightedByHuFuel.to_csv('../Output/borough_estimates_weightedByHuFuel.csv')

## Old code

In [8]:
## places cleanup
places['City'] = places['NAME'].str.replace(' city and borough', '').str.replace(' city', '')
places['City'] = places['NAME'].str.replace(' CDP', '').str.replace(' and borough', '').str.replace(' municipality', '')

In [9]:
## simData cleanup
## renaming
simData['City'] = simData['City'].str.replace('Juneau, Airport','Juneau').str.replace('Auke Bay','Juneau')
simData['City'] = simData['City'].str.replace('Douglas','Juneau').str.replace('Atqasuk - Res','Atqasuk')
simData['City'] = simData['City'].str.replace('Atqasuk - Com','Atwasuk').str.replace('Anaktuvuk Pass - C','Anaktuvuk Pass').str.replace('Anaktuvuk Pass Res','Anaktuvuk Pass')
simData['City'] = simData['City'].str.replace('/Utqiavik','').str.replace('Point Lay Commercial', 'Point Lay')
simData['City'] = simData['City'].str.replace('Chugiak', 'Anchorage').str.replace('Wainwright Com', 'Wainwright').str.replace('Point Hope Com','Pont Hope').str.replace('Point Hope Res', 'Point Hope')
simData['City'] = simData['City'].str.replace('Juneau, City of','Juneau').str.replace('Wainwright Res', 'Wainwright')
simData['City'] = simData['City'].str.replace('Clear', 'Healy Lake').str.replace("Denali Nat'l Park", 'Healy').str.replace('Dutch Habor', 'Unalaska')
simData['City'] = simData['City'].str.replace('Saint','St.').str.replace('Eyak','Cordova').str.replace('English Bay/Nanwal', 'Nanwalek').str.replace('Girdwood', 'Anchorage')
simData['City'] = simData['City'].str.replace('Sutton', 'Sutton-Alpine').str.replace('Indian', 'Anchorage').str.replace('Eagle River','Anchorage')
simData['City'] = simData['City'].str.replace('Bird Creek', 'Anchorage').str.replace('Naukati', 'Naukati Bay').str.replace('Faribanks - Chena Ridge', 'Fairbanks')

In [10]:
## code for merging simData annd  places in other to match houseunits with specific city
simHouse = simData.merge(places, on = 'City', how = 'left')
simHouse.drop(simHouse.columns[0], axis = 1, inplace = True)
simHouse

,City,Exist_Fuel_Type,Scenario,Census_Area,ANCSA_Region,Util_Name,PCE,Sq_Ft,Capital_Cost,Elec_Use_Jan,...,CO2_driving_miles_saved,Fuel_Use_Chg,Fuel_Price_Incremental,Elec_Use_Chg,Elec_Rate_Incremental,Econ,Exist_Fuel,Census_Area_LowerCase,NAME,HOUSEUNITS
0,Adak,1,Base,Aleutians West Census Area,Aleut,Adak -TDX Residential,0.7597,1130.0,6400.000000,471.936998,...,11917.022775,0.000000,NaN,-7085.615457,1.315811,Economic,Electricity,aleutians west census area,NaN,NaN
1,Adak,1,Small Load,Aleutians West Census Area,Aleut,Adak -TDX Residential,0.7597,678.0,6400.000000,377.549599,...,7150.178547,0.000000,NaN,-4251.348394,1.237769,Economic,Electricity,aleutians west census area,NaN,NaN
2,Adak,1,Large Load,Aleutians West Census Area,Aleut,Adak -TDX Residential,0.7597,1695.0,6400.000000,471.936998,...,17873.375506,0.000000,NaN,-10627.139693,1.325889,Economic,Electricity,aleutians west census area,NaN,NaN
3,Adak,1,Low Elec,Aleutians West Census Area,Aleut,Adak -TDX Residential,0.7597,1130.0,6400.000000,330.355899,...,11917.022775,0.000000,NaN,-7085.615457,1.315811,Economic,Electricity,aleutians west census area,NaN,NaN
4,Adak,1,No PCE,Aleutians West Census Area,Aleut,Adak -TDX Residential,0.7597,1130.0,6400.000000,471.936998,...,11917.022775,0.000000,NaN,-7085.615457,1.325889,Economic,Electricity,aleutians west census area,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3944,Yakutat,4,Small Load,Yakutat City and Borough,Sealaska Corporation,AVEC - Yakutat - Residential,0.3994,669.6,4498.730602,386.210227,...,688.842501,-181.179828,5.061,2268.388043,0.337302,Not Economic,#1 Oil,yakutat city and borough,Yakutat CDP,383.0
3945,Yakutat,4,Large Load,Yakutat City and Borough,Sealaska Corporation,AVEC - Yakutat - Residential,0.3994,1674.0,4498.730602,482.762784,...,3019.956559,-481.228472,5.061,5317.276317,0.551048,Not Economic,#1 Oil,yakutat city and borough,Yakutat CDP,383.0
3946,Yakutat,4,Low Elec,Yakutat City and Borough,Sealaska Corporation,AVEC - Yakutat - Residential,0.3994,1116.0,4498.730602,337.933949,...,1624.481897,-313.611895,5.061,3669.511105,0.367019,Close,#1 Oil,yakutat city and borough,Yakutat CDP,383.0
3947,Yakutat,4,No PCE,Yakutat City and Borough,Sealaska Corporation,AVEC - Yakutat - Residential,0.3994,1116.0,4498.730602,482.762784,...,1624.481897,-313.611895,5.061,3669.511105,0.576977,Not Economic,#1 Oil,yakutat city and borough,Yakutat CDP,383.0


In [12]:
## checking for missing houseunits
simHouse['HOUSEUNITS'].isnull().sum()

In [13]:
## code for getting NPV by Houseunits, storing it in a new dataframe: new_df
new_df = pd.DataFrame()
for u in simHouse['Census_Area'].unique():
    sub_df = simHouse[simHouse.Census_Area == u]
    sub_list = []
    for i in range(0,len(sub_df)):
        sub_list.append(sub_df.iloc[i]["NPV"]*(sub_df.iloc[i]["HOUSEUNITS"]/sum(sub_df["HOUSEUNITS"])))
    sub_df["new_NPV"] = sub_list
    new_df = pd.concat([new_df,sub_df])

<ipython-input-13-2e00151babc3>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df["new_NPV"] = sub_list
<ipython-input-13-2e00151babc3>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df["new_NPV"] = sub_list
<ipython-input-13-2e00151babc3>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#

In [14]:
## checking number of new_NPV that are missing
new_df['new_NPV'].isnull().sum()

3793

In [15]:
new_df

,City,Exist_Fuel_Type,Scenario,Census_Area,ANCSA_Region,Util_Name,PCE,Sq_Ft,Capital_Cost,Elec_Use_Jan,...,Fuel_Use_Chg,Fuel_Price_Incremental,Elec_Use_Chg,Elec_Rate_Incremental,Econ,Exist_Fuel,Census_Area_LowerCase,NAME,HOUSEUNITS,new_NPV
0,Adak,1,Base,Aleutians West Census Area,Aleut,Adak -TDX Residential,0.7597,1130.0,6400.000000,471.936998,...,0.000000,NaN,-7085.615457,1.315811,Economic,Electricity,aleutians west census area,NaN,NaN,NaN
1,Adak,1,Small Load,Aleutians West Census Area,Aleut,Adak -TDX Residential,0.7597,678.0,6400.000000,377.549599,...,0.000000,NaN,-4251.348394,1.237769,Economic,Electricity,aleutians west census area,NaN,NaN,NaN
2,Adak,1,Large Load,Aleutians West Census Area,Aleut,Adak -TDX Residential,0.7597,1695.0,6400.000000,471.936998,...,0.000000,NaN,-10627.139693,1.325889,Economic,Electricity,aleutians west census area,NaN,NaN,NaN
3,Adak,1,Low Elec,Aleutians West Census Area,Aleut,Adak -TDX Residential,0.7597,1130.0,6400.000000,330.355899,...,0.000000,NaN,-7085.615457,1.315811,Economic,Electricity,aleutians west census area,NaN,NaN,NaN
4,Adak,1,No PCE,Aleutians West Census Area,Aleut,Adak -TDX Residential,0.7597,1130.0,6400.000000,471.936998,...,0.000000,NaN,-7085.615457,1.325889,Economic,Electricity,aleutians west census area,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3944,Yakutat,4,Small Load,Yakutat City and Borough,Sealaska Corporation,AVEC - Yakutat - Residential,0.3994,669.6,4498.730602,386.210227,...,-181.179828,5.061,2268.388043,0.337302,Not Economic,#1 Oil,yakutat city and borough,Yakutat CDP,383.0,-182.041866
3945,Yakutat,4,Large Load,Yakutat City and Borough,Sealaska Corporation,AVEC - Yakutat - Residential,0.3994,1674.0,4498.730602,482.762784,...,-481.228472,5.061,5317.276317,0.551048,Not Economic,#1 Oil,yakutat city and borough,Yakutat CDP,383.0,-694.577764
3946,Yakutat,4,Low Elec,Yakutat City and Borough,Sealaska Corporation,AVEC - Yakutat - Residential,0.3994,1116.0,4498.730602,337.933949,...,-313.611895,5.061,3669.511105,0.367019,Close,#1 Oil,yakutat city and borough,Yakutat CDP,383.0,-61.814951
3947,Yakutat,4,No PCE,Yakutat City and Borough,Sealaska Corporation,AVEC - Yakutat - Residential,0.3994,1116.0,4498.730602,482.762784,...,-313.611895,5.061,3669.511105,0.576977,Not Economic,#1 Oil,yakutat city and borough,Yakutat CDP,383.0,-775.703007


In [18]:
## creating new column in heatData to merge on
heatData['Census_Area_LowerCase'] = heatData['Census_Area'].str.lower()

In [19]:
## merge new data with heatData
data = new_df.merge(heatData, on = 'Census_Area_LowerCase', how = 'left')

In [20]:
## NPV by heating source
newNPV = []
for i in range (0, len(data)):
    row = data.iloc[i]
    if row['Exist_Fuel_Type'] == 1:
        newNPV.append(row['new_NPV']*row['Electricity'])
    elif row['Exist_Fuel_Type'] == 2:
        newNPV.append(row['new_NPV']*row['Natural gas'])
    elif row['Exist_Fuel_Type'] == 3:
        newNPV.append(row['new_NPV']*row['Propane'])
    elif row['Exist_Fuel_Type'] == 4:
        newNPV.append(row['new_NPV']*row['Oil'])


In [21]:
## combine newNPV with data
data['newNPV'] = newNPV

In [22]:
data

,City,Exist_Fuel_Type,Scenario,Census_Area_x,ANCSA_Region,Util_Name,PCE,Sq_Ft,Capital_Cost,Elec_Use_Jan,...,Census_Area_LowerCase,NAME,HOUSEUNITS,new_NPV,Census_Area_y,Natural gas,Propane,Electricity,Oil,newNPV
0,Adak,1,Base,Aleutians West Census Area,Aleut,Adak -TDX Residential,0.7597,1130.0,6400.000000,471.936998,...,aleutians west census area,NaN,NaN,NaN,Aleutians West Census Area,0.012357,0.006178,0.060900,0.920565,NaN
1,Adak,1,Small Load,Aleutians West Census Area,Aleut,Adak -TDX Residential,0.7597,678.0,6400.000000,377.549599,...,aleutians west census area,NaN,NaN,NaN,Aleutians West Census Area,0.012357,0.006178,0.060900,0.920565,NaN
2,Adak,1,Large Load,Aleutians West Census Area,Aleut,Adak -TDX Residential,0.7597,1695.0,6400.000000,471.936998,...,aleutians west census area,NaN,NaN,NaN,Aleutians West Census Area,0.012357,0.006178,0.060900,0.920565,NaN
3,Adak,1,Low Elec,Aleutians West Census Area,Aleut,Adak -TDX Residential,0.7597,1130.0,6400.000000,330.355899,...,aleutians west census area,NaN,NaN,NaN,Aleutians West Census Area,0.012357,0.006178,0.060900,0.920565,NaN
4,Adak,1,No PCE,Aleutians West Census Area,Aleut,Adak -TDX Residential,0.7597,1130.0,6400.000000,471.936998,...,aleutians west census area,NaN,NaN,NaN,Aleutians West Census Area,0.012357,0.006178,0.060900,0.920565,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3944,Yakutat,4,Small Load,Yakutat City and Borough,Sealaska Corporation,AVEC - Yakutat - Residential,0.3994,669.6,4498.730602,386.210227,...,yakutat city and borough,Yakutat CDP,383.0,-182.041866,Yakutat City and Borough,0.020408,0.000000,0.010204,0.969388,-176.469156
3945,Yakutat,4,Large Load,Yakutat City and Borough,Sealaska Corporation,AVEC - Yakutat - Residential,0.3994,1674.0,4498.730602,482.762784,...,yakutat city and borough,Yakutat CDP,383.0,-694.577764,Yakutat City and Borough,0.020408,0.000000,0.010204,0.969388,-673.315179
3946,Yakutat,4,Low Elec,Yakutat City and Borough,Sealaska Corporation,AVEC - Yakutat - Residential,0.3994,1116.0,4498.730602,337.933949,...,yakutat city and borough,Yakutat CDP,383.0,-61.814951,Yakutat City and Borough,0.020408,0.000000,0.010204,0.969388,-59.922656
3947,Yakutat,4,No PCE,Yakutat City and Borough,Sealaska Corporation,AVEC - Yakutat - Residential,0.3994,1116.0,4498.730602,482.762784,...,yakutat city and borough,Yakutat CDP,383.0,-775.703007,Yakutat City and Borough,0.020408,0.000000,0.010204,0.969388,-751.956996
